In [1]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
import pandas as pd

## Part 1: Income Process

In [2]:
def unc_mean(x):
    y_h, y_l = x
    return(y_h/2 + y_l/2)

def unc_variance(x):
    y_h, y_l = x
    return(np.log(y_h)**2/2 + np.log(y_l)**2/2 - (np.log(y_h)/2 + np.log(y_l)/2)**2)

def system_1(x):
    return(unc_mean(x) - 1, unc_variance(x) - 0.02**2)

y_h, y_l = opt.fsolve(system_1, (1,0.5))

def auto_correl(x):
    return(((1-x)*(np.log(y_h)**2 + np.log(y_l)**2)/2 + x*np.log(y_h)*np.log(y_l) 
            - (np.log(y_h)/2 + np.log(y_l)/2)**2)/unc_variance((y_h,y_l)) - 0.878)

pi = opt.fsolve(auto_correl, 0)
pi = pi[0]

P = np.array([(1-pi,pi),(pi,1-pi)])

We find the following values for the income process:
$$ y_h = 1.02  $$
$$ y_l = 0.98 $$
$$ \pi = 0.06 $$ 

## Part 2: Steady State and Calibration

In [3]:
# Parameters:
beta = 0.99
s = 0.1
alpha = 0.72
eta = 1 - alpha
kappa = 1
y_ss = 1

# Equilibrium Conditions:
def ss_vacancies(x):
    theta, b, h = x
    return(kappa - beta*h*theta**(-alpha)*eta*(y_ss - b)/(1-beta*(1-s-h*theta**(1-alpha)*(1-eta))))

def ss_fr(x):
    theta, b, h = x
    return(h*theta**(1-alpha))

def ss_ratio(x):
    theta, b, h = x
    return((y_ss - (1-beta*(1-s))*eta*((y_ss - b)/(1-beta*(1-s-h*theta**(1-alpha)*(1-eta)))))/b)

def system_2(x):
    return(ss_vacancies(x),ss_fr(x) - 0.83, ss_ratio(x) - 1/0.4)

theta_ss, b, h = opt.fsolve(system_2, (0.2,0.3,1)) 
u_ss = s/(s+h*theta_ss**(1-alpha))

We find the following:
$$\theta_{ss} = 0.2005 $$
$$ b = 0.3894 $$
$$ h = 1.3016 $$

We can normalize $\kappa = 1$ without loss of generality because the matching function exhibits constant returns to scale.

## Part 3: Solving for $\Theta(y_h)$ and $\Theta(y_l)$:

In [4]:
tol = 1e-9
diff = 1
income_supp = np.array([y_h,y_l]) 

S_init = income_supp - b

while diff > tol:
    theta = ((h*beta*eta/kappa)*S_init@P)**(1/alpha)
    S_new = income_supp - b + beta*(1-s - h*theta**(1-alpha)*(1-eta))*(S_init@P)
    diff = np.amax(abs(S_new - S_init))
    S_init = S_new


We find the following values for $\Theta(\cdot)$:
$$ \Theta(y_h) = 0.2065 $$
$$ \Theta(y_l) = 0.1946  $$

## Part 4: Standard Deviations and Autocorrelations

In [5]:
# Simulate State of the System:
N = 20000
states = ('H','L')
state_sim = [states[0]]
for it in range(N):
    if state_sim[it] == 'H':
        state_sim.append(np.random.choice(states,1, p = [1-pi,pi])[0])
    else:
        state_sim.append(np.random.choice(states,1, p = [pi,1-pi])[0])
        

In [6]:
# Unemployment:
u_sim = [u_ss]
for it in range(N):
    if state_sim[it] == 'H':
        u_sim.append((1-h*theta[0]**(1-alpha))*u_sim[it] + (1-u_sim[it])*s )
    else:
        u_sim.append((1-h*theta[1]**(1-alpha))*u_sim[it] + (1-u_sim[it])*s )
        
# Vacancies
v_ss = theta_ss*u_ss
v_sim = [v_ss]
for it in range(N):
    if state_sim[it] == 'H':
        v_sim.append(theta[0]*u_sim[it])
    else:
        v_sim.append(theta[1]*u_sim[it])

# Market Tightness
theta_sim = [theta_ss]
for it in range(N):
    if state_sim[it] == 'H':
        theta_sim.append(theta[0])
    else:
        theta_sim.append(theta[1])
        
# Finding Rate:
fr_ss = h*theta_ss**(1-alpha)
fr_sim = [fr_ss]
for it in range(N):
    if state_sim[it] == 'H':
        fr_sim.append(h*theta[0]**(1-alpha))
    else:
        fr_sim.append(h*theta[1]**(1-alpha))


u_sim = np.log(u_sim)
v_sim = np.log(v_sim)
theta_sim = np.log(theta_sim)
fr_sim = np.log(fr_sim)

u_sd = np.sqrt(np.var(u_sim))
u_acr = pd.Series(u_sim).autocorr(4)

v_sd = np.sqrt(np.var(v_sim))
v_acr = pd.Series(v_sim).autocorr(4)

theta_sd = np.sqrt(np.var(theta_sim))
theta_acr = pd.Series(theta_sim).autocorr(4)

fr_sd = np.sqrt(np.var(fr_sim))
fr_acr = pd.Series(fr_sim).autocorr(4)


We find the following values for the standard deviation and quarterly autocorrelations of unemployment, vacancies, market tightness, and finding rate:
$$ \sigma_u = 0.0074 \hspace{0.8cm} \rho_u = 0.6143 $$
$$ \sigma_v = 0.0235 \hspace{0.8cm} \rho_v = 0.5394 $$
$$ \sigma_\theta = 0.0298 \hspace{0.8cm} \rho_\theta = 0.6035 $$
$$ \sigma_f = 0.0083 \hspace{0.8cm} \rho_f = 0.6035 $$


## Part 5: Changing Unemployment Benefits

In [18]:
b = 0.5

tol = 1e-9
diff = 1
income_supp = np.array([y_h,y_l]) 

S_init = income_supp - b

while diff > tol:
    theta = ((h*beta*eta/kappa)*S_init@P)**(1/alpha)
    S_new = income_supp - b + beta*(1-s - h*theta**(1-alpha)*(1-eta))*(S_init@P)
    diff = np.amax(abs(S_new - S_init))
    S_init = S_new

With this new value for $b$, we have the following values for $\Theta(\cdot)$:
$$ \Theta(y_h) = 0.1686 $$
$$ \Theta(y_l) = 0.1568 $$
As expected, higher value of unemployment benefits decreases market tightness since this lowers the surplus from a match for a firm and increases the value of unemployment for workers. 

In [19]:
# Simulate State of the System:
N = 20000
states = ('H','L')
state_sim = [states[0]]
for it in range(N):
    if state_sim[it] == 'H':
        state_sim.append(np.random.choice(states,1, p = [1-pi,pi])[0])
    else:
        state_sim.append(np.random.choice(states,1, p = [pi,1-pi])[0])
        
# Unemployment:
u_sim = [u_ss]
for it in range(N):
    if state_sim[it] == 'H':
        u_sim.append((1-h*theta[0]**(1-alpha))*u_sim[it] + (1-u_sim[it])*s )
    else:
        u_sim.append((1-h*theta[1]**(1-alpha))*u_sim[it] + (1-u_sim[it])*s )
        
# Vacancies
v_ss = theta_ss*u_ss
v_sim = [v_ss]
for it in range(N):
    if state_sim[it] == 'H':
        v_sim.append(theta[0]*u_sim[it])
    else:
        v_sim.append(theta[1]*u_sim[it])

# Market Tightness
theta_sim = [theta_ss]
for it in range(N):
    if state_sim[it] == 'H':
        theta_sim.append(theta[0])
    else:
        theta_sim.append(theta[1])
        
# Finding Rate:
fr_ss = h*theta_ss**(1-alpha)
fr_sim = [fr_ss]
for it in range(N):
    if state_sim[it] == 'H':
        fr_sim.append(h*theta[0]**(1-alpha))
    else:
        fr_sim.append(h*theta[1]**(1-alpha))


u_sim = np.log(u_sim)
v_sim = np.log(v_sim)
theta_sim = np.log(theta_sim)
fr_sim = np.log(fr_sim)

u_sd = np.sqrt(np.var(u_sim))
u_acr = pd.Series(u_sim).autocorr(4)

v_sd = np.sqrt(np.var(v_sim))
v_acr = pd.Series(v_sim).autocorr(4)

theta_sd = np.sqrt(np.var(theta_sim))
theta_acr = pd.Series(theta_sim).autocorr(4)

fr_sd = np.sqrt(np.var(fr_sim))
fr_acr = pd.Series(fr_sim).autocorr(4)

Moreover, higher value of $b$ should increase the elasticity of market tightness, so we should find more volatility. These are the standard deviations and the autocorrelations following the increase in unemployment benefits:
$$ \sigma_u = 0.0089 \hspace{0.8cm} \rho_u = 0.6093 $$
$$ \sigma_v = 0.0288 \hspace{0.8cm} \rho_v = 0.5217 $$
$$ \sigma_\theta = 0.0363 \hspace{0.8cm} \rho_\theta = 0.5901 $$
$$ \sigma_f = 0.0102\hspace{0.8cm} \rho_f = 0.5901 $$
As it can be seen, there is indeed an increase in volatilities and a decrease in persistence. This decrease in autocorrelation follows from the fact that...